In [ ]:
# Install required libs
# !pip install -U segmentation-models-pytorch albumentations --user 

## Loading data

In [ ]:
import os
import sys
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import cv2
import matplotlib.pyplot as plt
from traversability_estimation.utils import visualize
from traversability_estimation.rellis_3d import DatasetSemSeg as Dataset
from torch.utils.data import DataLoader
import albumentations as albu
import torch
import numpy as np
import segmentation_models_pytorch as smp

### Dataloader

Writing helper class for data extraction, tranformation and preprocessing  
https://pytorch.org/docs/stable/data

In [ ]:
# Lets look at data we have

# ds = Dataset(classes=['grass', 'tree', 'sky'], split='train')
# ds = Dataset(classes=['grass', 'tree', 'sky'], split='val')
ds = Dataset(classes=['grass', 'tree', 'sky'], split='test')

ind = int( np.random.choice(range(len(ds))) )
image, mask = ds[ind] # get some sample
image_vis = image * ds.std + ds.mean

visualize(
    image=image_vis, 
    grass_mask=mask[0, ...],
    tree_mask=mask[1, ...],
    sky_mask=mask[2, ...]
)

### Augmentations

Data augmentation is a powerful technique to increase the amount of your data and prevent model overfitting.  
If you not familiar with such trick read some of these articles:
 - [The Effectiveness of Data Augmentation in Image Classification using Deep
Learning](http://cs231n.stanford.edu/reports/2017/pdfs/300.pdf)
 - [Data Augmentation | How to use Deep Learning when you have Limited Data](https://medium.com/nanonets/how-to-use-deep-learning-when-you-have-limited-data-part-2-data-augmentation-c26971dc8ced)
 - [Data Augmentation Experimentation](https://towardsdatascience.com/data-augmentation-experimentation-3e274504f04b)

Since our dataset is very small we will apply a large number of different augmentations:
 - horizontal flip
 - affine transforms
 - perspective transforms
 - brightness/contrast/colors manipulations
 - image bluring and sharpening
 - gaussian noise
 - random crops

All this transforms can be easily applied with [**Albumentations**](https://github.com/albu/albumentations/) - fast augmentation library.
For detailed explanation of image transformations you can look at [kaggle salt segmentation exmaple](https://github.com/albu/albumentations/blob/master/notebooks/example_kaggle_salt.ipynb) provided by [**Albumentations**](https://github.com/albu/albumentations/) authors.

In [ ]:
src_size = np.array(image.shape[:2])
src_size

In [ ]:
# Visualize resulted augmented images and masks

ds_aug = Dataset(split='train')

# same image with different random transforms
for i in range(3):
    image, mask = ds_aug[1]
    image_vis = image.transpose([1, 2, 0]) * ds_aug.std + ds_aug.mean
    
    visualize(image=image_vis, grass_mask=mask[2, ...])

## Create model and train

In [ ]:
ENCODER = 'se_resnext50_32x4d'
ENCODER_WEIGHTS = 'imagenet'
CLASSES = ['void', 'dirt', 'grass', 'tree', 'pole', 'water',
           'sky', 'vehicle', 'object', 'asphalt', 'building',
           'log', 'person', 'fence', 'bush', 'concrete',
           'barrier', 'puddle', 'mud', 'rubble']
ACTIVATION = 'sigmoid' if len(CLASSES) == 1 else 'softmax2d'  # could be None for logits or 'softmax2d' for multiclass segmentation
DEVICE = 'cuda'
IMG_SIZE = (352, 640)
LR = 0.0001
# create segmentation model with pretrained encoder
model = smp.FPN(
    encoder_name=ENCODER, 
    encoder_weights=ENCODER_WEIGHTS, 
    classes=len(CLASSES),
    activation=ACTIVATION,
)

In [ ]:
train_dataset = Dataset(classes=CLASSES, crop_size=IMG_SIZE, split='train')
valid_dataset = Dataset(classes=CLASSES, crop_size=IMG_SIZE, split='val')


train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=12)
valid_loader = DataLoader(valid_dataset, batch_size=1, shuffle=False, num_workers=4)

In [ ]:
# Dice/F1 score - https://en.wikipedia.org/wiki/S%C3%B8rensen%E2%80%93Dice_coefficient
# IoU/Jaccard score - https://en.wikipedia.org/wiki/Jaccard_index

loss = smp.utils.losses.DiceLoss()
metrics = [
    smp.utils.metrics.IoU(threshold=0.5),
]

optimizer = torch.optim.Adam([ 
    dict(params=model.parameters(), lr=LR),
])

In [ ]:
# create epoch runners 
# it is a simple loop of iterating over dataloader`s samples
train_epoch = smp.utils.train.TrainEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    optimizer=optimizer,
    device=DEVICE,
    verbose=True,
)

valid_epoch = smp.utils.train.ValidEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    device=DEVICE,
    verbose=True,
)

In [ ]:
# train model

max_score = 0
n_epochs = 10
for i in range(0, n_epochs):
    
    print('\nEpoch: {}'.format(i))
    train_logs = train_epoch.run(train_loader)
    valid_logs = valid_epoch.run(valid_loader)
    
    # do something (save model, change lr, etc.)
    if max_score < valid_logs['iou_score']:
        max_score = valid_logs['iou_score']
        torch.save(model, './best_model.pth')
        print('Model saved!')
        
    if i == 25:
        optimizer.param_groups[0]['lr'] = 1e-5
        print('Decrease decoder learning rate to 1e-5!')

## Test best saved model

In [ ]:
# load best saved checkpoint
# best_model = torch.load('./best_model.pth')
best_model = torch.load('../config/weights/smp/se_resnext50_32x4d_352x640_lr1e-4.pth')
# best_model = torch.load('../config/weights/smp/mobilenet_v2_352x640_lr1e-4_iou_0.46.pth')

## Visualize predictions

In [ ]:
from hrnet.core.function import convert_label, convert_color
import yaml

test_dataset = Dataset(classes=CLASSES, crop_size=(704, 960), split='test')

cfg = yaml.safe_load(open('../config/rellis.yaml', 'r'))

def mask_to_colormap(mask, cfg):
    mask = np.argmax(mask, axis=0).astype(np.uint8)
    mask = convert_label(mask, True)
    mask = convert_color(mask, cfg["color_map"])
    return mask


for i in range(1):
    n = np.random.choice(len(test_dataset))
    
    image, gt_mask = test_dataset[n]
    image_vis = image * test_dataset.std + test_dataset.mean
    
    gt_mask = gt_mask.squeeze()
    
    x_tensor = torch.from_numpy(image.transpose([2, 0, 1])).to(DEVICE).unsqueeze(0)
    pr_mask = best_model.predict(x_tensor)
    pr_mask = (pr_mask.squeeze().cpu().numpy().round())
    
    pred_colormap = mask_to_colormap(pr_mask, cfg)
    gt_colormap = mask_to_colormap(gt_mask, cfg)

    visualize(image=image_vis, pred=pred_colormap, gt=gt_colormap)